In [ ]:
cd ..

In [ ]:
import os
import pandas as pd
import numpy as np
from src.analyzer import DataAnalyzer, plot_fill_between
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
output_dir = 'output/motion_benefit3'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.pkl')]
pkl_fns.sort()
len(pkl_fns)

In [ ]:
out_ = []
# for da in da_:
for pkl_fn in pkl_fns:
    da = DataAnalyzer.fromfilename(pkl_fn)
    out = []
    out.append(da.data['EM_data']['mode']) # Eye path given or not
    out.append(da.data['motion_gen']['mode']) # True eye movements or no eye movements
    try:
        out.append(da.data['motion_gen']['dc'])
    except KeyError:
        out.append(0.0)
    out.append(da.data['motion_prior']['dc'])
    out.append(da.data['ds']) # Image size
    out = out + da.snr_list()
    out_.append(out)

In [ ]:
t = da.time_list()
data = pd.DataFrame.from_records(
    out_, columns=['inference_mode', 'gen_mode', 'gen_dc', 'prior_dc', 'ds'] + list(t))
grouped = pd.groupby(data, ['inference_mode', 'gen_mode', 'gen_dc', 'ds'])
c_ = plt.cm.rainbow(np.linspace(0, 1, len(grouped)))
np.random.shuffle(c_)
c_ = list(c_)
len(grouped)

In [ ]:
for name, group in grouped:
    mode, gen_mode, gen_dc, ds = name
    if ds != 0.4:
        continue
    print name
    print group[list(t)[-1]].mean()

In [ ]:
plt.figure(figsize=(10, 7))
plt.title('SNR as a function of time')
alpha = 0.75
for c, (name, group) in zip(c_, grouped):
    inf_mode, gen_mode, gen_dc, ds = name
#     if mode != 'EM':
#         continue
    if ds != .40:
        continue
    label = 'Inf: {} Gen: {} GenDC: {} ds: {:.2f}'.format(*name)
#     plot_fill_between(t, group[list(t)], label=label, c=c, k=0.5)
    plt.plot(t, group[list(t)].T.iloc[:, 0], c=c, label=label, alpha=alpha)
    plt.plot(t, group[list(t)].T.iloc[:, 1:], c=c, alpha=alpha);
    plt.xlabel('time (ms)')
plt.legend(loc='upper left')
plt.ylim([0, 12]);
# plt.savefig(os.path.join(output_dir, 'motion_benefit.png'), dpi=200)

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
final_snrs = [group[list(t)[-1]].values for k, group in grouped]

ks_2samp(final_snrs[1], final_snrs[4])

In [ ]:
idx = data[(data['ds'] == 0.40).values].index

In [ ]:
da_ = [DataAnalyzer.fromfilename(pkl_fn) for pkl_fn in [pkl_fns[i] for i in idx]]

In [ ]:
da = da_[24]
t = da.time_list()
da.data['EM_data']['mode'], da.data['ds']

In [ ]:
da.data['de']

In [ ]:
plt.figure(figsize=(5, 5))
da.plot_image_and_rfs()

plt.title('E Projected onto the Retina \n E thickness: {:.2f} arcmin \n Neuron Spacing {:.2f} arcmin'.format(
        2 * da.data['ds'], da.data['de']))
a = 6; plt.xlim([-a, a]); plt.ylim([-a, a])

In [ ]:
da.plot_em_estimate(-1)

In [ ]:
ax = plt.axes()
da.plot_image_estimate(-1)
ax.set_title('')
ax.

In [ ]:
plt.figure(figsize=(4 * 2, 3.5 * 5))
for i in range(10):
    plt.subplot(5, 2, i + 1)
    da_[i].plot_image_estimate(-1)

In [ ]:
plt.figure(figsize=(4 * 2, 3.5 * 5))
for i in range(20, 30):
    plt.subplot(5, 2, i - 20 + 1)
    da_[i].plot_image_estimate(-1)

In [ ]:
for i, t in enumerate([0, 10, 20, 80-1]):
    plt.subplot(2, 2, i + 1)
    da.plot_image_estimate(t)

# Group SNR plots by image size and diffusion constant

In [ ]:
out_ = []
for pkl_fn in pkl_fns:
    da = DataAnalyzer.fromfilename(pkl_fn)
    out = [da.data['ds'], da.data['motion_gen']['dc']]
    out = out + da.SNR_list()
    out_.append(out)

In [ ]:
t = da_[0].time_list()

In [ ]:
data = pd.DataFrame.from_records(out_, columns=['ds', 'dc_gen'] + list(t))
# data = data[data['ds'] == 0.5]

In [ ]:
grouped = pd.groupby(data, ['ds', 'dc_gen'])
c_ = ['g', 'b', 'r', 'y', 'm', 'c']

In [ ]:
plt.figure(figsize=(10, 7))
plt.title('SNR as a function of time')
for c, (name, group) in zip(c_, grouped):
    label = 'ds={}, dc={}'.format(*name)
    plot_fill_between(t, group[list(t)], label=label, c=c)
plt.legend(loc='upper left')
# plt.savefig(os.path.join(output_dir, 'motion_benefit.png'), dpi=200)